In [1]:
!pip install pandas scikit-learn joblib sqlalchemy psycopg2-binary

In [2]:
import pandas as pd
import joblib
import logging
from sqlalchemy import create_engine
from sklearn.ensemble import IsolationForest

In [3]:
# Konfigurasi logging untuk melihat output
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = logging.getLogger(__name__)

In [4]:
# Menghubungkan ke database untuk mengambil data historis sebagai data latih.

DB_USER = 'user'
DB_PASSWORD = 'password'
DB_HOST = 'postgres'
DB_PORT = '5432'
DB_NAME = 'machine_db'

In [5]:
db_connection_str = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
try:
    engine = create_engine(db_connection_str)
    log.info("Berhasil terhubung ke database PostgreSQL.")
except Exception as e:
    log.error(f"Gagal terhubung ke database: {e}")
    engine = None

2025-07-14 13:04:04,597 - INFO - Berhasil terhubung ke database PostgreSQL.


In [6]:
# Mengambil Data Latih (Hanya Data dengan Kondisi Normal)
if engine:
    try:
        # Query untuk mengambil semua data di mana 'label' adalah 0 (Normal)
        query = "SELECT * FROM sensor_readings WHERE label = 0"
        
        # Membaca data dari database ke dalam DataFrame Pandas
        df_normal = pd.read_sql(query, engine)
        
        log.info(f"Berhasil mengambil {len(df_normal)} baris data 'Normal' untuk pelatihan.")
        print(df_normal.head())
        
    except Exception as e:
        log.error(f"Gagal mengambil data: {e}")
        df_normal = None
else:
    log.warning("Koneksi database tidak tersedia.")

2025-07-14 13:04:09,150 - INFO - Berhasil mengambil 1598 baris data 'Normal' untuk pelatihan.


   id           event_timestamp machine_id  vibration  acoustic  temperature  \
0   1 2024-07-01 08:00:00+00:00        M01      0.822     0.645        66.85   
1   3 2024-07-01 08:02:00+00:00        M01      0.856     0.590        67.03   
2   4 2024-07-01 08:03:00+00:00        M01      0.793     0.544        65.04   
3   6 2024-07-01 08:05:00+00:00        M01      0.782     0.655        61.95   
4   7 2024-07-01 08:06:00+00:00        M01      0.837     0.564        64.52   

   current  status  label  imf_1  imf_2  imf_3  \
0    13.04  Normal      0  0.196  0.033  0.000   
1    12.30  Normal      0  0.187  0.017  0.002   
2    11.69  Normal      0  0.196 -0.060  0.003   
3    11.56  Normal      0  0.164 -0.081  0.005   
4    10.98  Normal      0  0.238 -0.095  0.006   

                   processing_time  
0 2025-07-10 16:21:08.843890+00:00  
1 2025-07-10 16:21:08.879127+00:00  
2 2025-07-10 16:21:08.888492+00:00  
3 2025-07-10 16:21:08.899494+00:00  
4 2025-07-10 16:21:08.919060+00:0

In [7]:
if df_normal is not None and not df_normal.empty:
    feature_columns = [
        'vibration', 
        'acoustic', 
        'temperature', 
        'current', 
        'imf_1', 
        'imf_2', 
        'imf_3'
    ]

    # Membuat DataFrame fitur (X)
    X_train_normal = df_normal[feature_columns]
    
    log.info("Fitur untuk pelatihan model telah disiapkan.")
    print(X_train_normal.describe())
else:
    log.warning("Tidak ada data latih untuk diproses.")

2025-07-14 13:04:48,045 - INFO - Fitur untuk pelatihan model telah disiapkan.


         vibration     acoustic  temperature      current        imf_1  \
count  1598.000000  1598.000000  1598.000000  1598.000000  1598.000000   
mean      0.798652     0.600558    64.959819    11.989587     0.160327   
std       0.049571     0.049436     2.046579     0.505777     0.047844   
min       0.645000     0.430000    58.360000    10.290000     0.073000   
25%       0.765000     0.567000    63.520000    11.660000     0.118000   
50%       0.798000     0.602500    65.060000    12.000000     0.159000   
75%       0.831000     0.634000    66.317500    12.320000     0.201000   
max       0.979000     0.772000    71.360000    13.810000     0.265000   

             imf_2        imf_3  
count  1598.000000  1598.000000  
mean     -0.000345     0.000986  
std       0.068327     0.035928  
min      -0.136000    -0.050000  
25%      -0.056000    -0.036000  
50%      -0.002000     0.004000  
75%       0.056000     0.037000  
max       0.143000     0.050000  


In [8]:
# Latih Model Deteksi Anomali (Isolation Forest)

if 'X_train_normal' in locals() and not X_train_normal.empty:
    log.info("Memulai pelatihan model Isolation Forest...")
    
    # Inisialisasi model
    # 'contamination' diatur ke 'auto' agar algoritma bisa menentukan ambang batas anomali secara otomatis.
    model_anomaly = IsolationForest(
        n_estimators=100, 
        contamination='auto', 
        random_state=42
    )
    
    # Melatih model hanya dengan data normal
    model_anomaly.fit(X_train_normal)
    
    log.info("Pelatihan model selesai.")
else:
    log.warning("Tidak ada data untuk melatih model.")

2025-07-14 13:05:05,471 - INFO - Memulai pelatihan model Isolation Forest...
2025-07-14 13:05:05,836 - INFO - Pelatihan model selesai.


In [12]:
# Simpan Model yang Sudah Dilatih
# Menyimpan model ke dalam sebuah file agar bisa dimuat dan digunakan
# oleh proses streaming tanpa perlu melatih ulang.

if 'model_anomaly' in locals():
    file_path = 'model/anomaly_detection_model.pkl'
    try:
        joblib.dump(model_anomaly, file_path)
        log.info(f"Model berhasil disimpan ke: {file_path}")
    except Exception as e:
        log.error(f"Gagal menyimpan model: {e}")
else:
    log.warning("Model tidak tersedia untuk disimpan.")

2025-07-14 13:07:15,323 - INFO - Model berhasil disimpan ke: model/anomaly_detection_model.pkl
